In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Read the CSV File that contains the stock and sectors

In [2]:
NIFTY50_name = pd.read_csv("D:\\capstone_project\\capstone_project\\NIFTY50_category.csv")

# Load the original information of stock price

In [10]:
NIFTY50_stock = {}
for target in NIFTY50_name["company"]:
    da = {}
    da['category'] = NIFTY50_name[NIFTY50_name.company == target]['category'].iloc[0]
    try:
        da['stock_price'] = pd.read_csv(f"D:\\capstone_project\\capstone_project\\NIFTY50\\SP500_dataset\\{target}.csv")
    except FileNotFoundError:
        print(f"File not found for {target}. Skipping...")
        continue
    
    if 'Date ' not in da['stock_price'].columns:
        print(f"'Date' column not found in {target}.csv. Skipping...")
        continue

    NIFTY50_stock[target] = da

In [12]:
type(NIFTY50_name.iloc[20]['company'])

str

In [9]:
NIFTY50_stock.keys()

dict_keys(['HINDUNILVR', 'NESTLEIND', 'HEROMOTOCO', 'BRITANNIA', 'TATACONSUM', 'CIPLA', 'TCS', 'APOLLOHOSP', 'TECHM', 'ASIANPAINT', 'LTIM', 'TITAN', 'DIVISLAB', 'BAJAJ-AUTO', 'HCLTECH', 'INFY', 'DRREDDY', 'WIPRO', 'SUNPHARMA', 'ITC', 'MARUTI', 'M&M', 'HDFCLIFE', 'SBILIFE', 'EICHERMOT', 'KOTAKBANK', 'GRASIM', 'TATAMOTORS', 'ULTRACEMCO', 'HDFCBANK', 'BAJFINANCE', 'BAJAJFINSV', 'HINDALCO', 'BHARTIARTL', 'ICICIBANK', 'RELIANCE', 'AXISBANK', 'JSWSTEEL', 'TATASTEEL', 'INDUSINDBK', 'SHRIRAMFIN', 'POWERGRID', 'LT', 'BPCL', 'SBIN', 'COALINDIA', 'NTPC', 'ONGC', 'ADANIENT', 'ADANIPORTS'])

# Load every stock and its data into a dictionary file

In [8]:
for i in NIFTY50_stock.keys():
    dd = pd.DataFrame()
    dd['Date '] = NIFTY50_stock[i]['stock_price']['Date ']
    dd['OPEN '] = NIFTY50_stock[i]['stock_price']['OPEN ']
    dd['HIGH '] = NIFTY50_stock[i]['stock_price']['HIGH ']
    dd['LOW '] = NIFTY50_stock[i]['stock_price']['LOW ']
    dd['PREV. CLOSE '] = NIFTY50_stock[i]['stock_price']['PREV. CLOSE ']
    dd['close '] = NIFTY50_stock[i]['stock_price']['close ']
    NIFTY50_stock[i]['stock_price'] =  dd

In [9]:
NIFTY50_stock['ONGC']['stock_price']

,Date,OPEN,HIGH,LOW,PREV. CLOSE,close
0,26-Jun-2024,266.90,269.15,264.10,267.00,267.75
1,25-Jun-2024,270.10,270.80,265.00,269.90,267.00
2,24-Jun-2024,269.65,271.25,266.60,269.65,269.90
3,21-Jun-2024,271.45,274.75,268.90,271.85,269.65
4,20-Jun-2024,271.80,274.00,269.10,271.55,271.85
...,...,...,...,...,...,...
492,01-Jul-2022,148.95,151.15,130.00,151.55,131.05
493,30-Jun-2022,153.00,154.45,149.25,154.15,151.55
494,29-Jun-2022,149.35,157.40,146.20,149.35,154.15
495,28-Jun-2022,142.80,151.75,142.15,141.50,149.35


# Check if all stocks have the same dates

In [11]:
need_day = np.array(NIFTY50_stock["ONGC"]["stock_price"]['Date '])
for target in NIFTY50_stock.keys():
    if 'Date ' in NIFTY50_stock[target]["stock_price"].columns:
        NIFTY50_stock[target]["stock_price"] = NIFTY50_stock[target]["stock_price"][
            NIFTY50_stock[target]["stock_price"]['Date '].isin(need_day)].reset_index(drop=True)
        NIFTY50_stock[target]["stock_price"].index = NIFTY50_stock[target]["stock_price"]['Date ']
    else:
        print(f"'Date' column missing for {target}. Skipping...")

In [16]:
NIFTY50_stock['ONGC']['stock_price']

,Date,series,OPEN,HIGH,LOW,PREV. CLOSE,ltp,close,vwap,52W H,52W L,VOLUME,VALUE,No of trades
Date,,,,,,,,,,,,,,
26-Jun-2024,26-Jun-2024,EQ,266.90,269.15,264.10,267.00,267.8,267.75,266.57,292.95,155.85,"1,06,50,910","2,83,92,00,588.35","1,05,797"
25-Jun-2024,25-Jun-2024,EQ,270.10,270.80,265.00,269.90,267.1,267.00,267.16,292.95,155.85,"1,44,25,434","3,85,39,15,651.60","1,19,470"
24-Jun-2024,24-Jun-2024,EQ,269.65,271.25,266.60,269.65,270.1,269.90,269.22,292.95,155.85,"72,82,347","1,96,05,31,156.75","74,981"
21-Jun-2024,21-Jun-2024,EQ,271.45,274.75,268.90,271.85,270.0,269.65,271.26,292.95,155.40,"1,74,06,416","4,72,16,50,927.75","1,05,432"
20-Jun-2024,20-Jun-2024,EQ,271.80,274.00,269.10,271.55,272.2,271.85,271.96,292.95,155.40,"1,28,26,509","3,48,83,16,117.15","1,45,663"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
01-Jul-2022,01-Jul-2022,EQ,148.95,151.15,130.00,151.55,131.4,131.05,136.30,194.95,108.50,"12,57,88,305","17,14,43,26,289.40","5,09,039"
30-Jun-2022,30-Jun-2022,EQ,153.00,154.45,149.25,154.15,150.9,151.55,151.62,194.95,108.50,"3,87,25,546","5,87,16,30,059.15","1,85,317"
29-Jun-2022,29-Jun-2022,EQ,149.35,157.40,146.20,149.35,153.6,154.15,152.82,194.95,108.50,"16,58,60,322","25,34,68,03,135.95","2,72,601"


# Normalize stock price, check if the 'close ' column is of type string, if not convert it to string, remove commas and convert to float

In [17]:
normalize_scalar = {}
for target in NIFTY50_stock.keys():
    scaler = StandardScaler()
    if NIFTY50_stock[target]["stock_price"]["close "].dtype != 'object':
        NIFTY50_stock[target]["stock_price"]["close "] = NIFTY50_stock[target]["stock_price"]["close "].astype(str)
    NIFTY50_stock[target]["stock_price"]["close "] = NIFTY50_stock[target]["stock_price"]["close "].str.replace(',', '').astype(float)
    nor_data = scaler.fit_transform(np.array(NIFTY50_stock[target]["stock_price"]["close "]).reshape(-1, 1)).ravel()
    NIFTY50_stock[target]["stock_price"]["nor_close"] = nor_data
    normalize_scalar[target] = scaler

# Calculate return ratio

In [18]:
for target in NIFTY50_stock.keys():
    return_tratio = []
    data = np.array(NIFTY50_stock[target]["stock_price"]["close "])
    for i in range(len(data)):
        if i == 0:
            return_tratio.append(0)
        else:
            return_tratio.append((data[i] - data[i - 1]) / data[i - 1])
    NIFTY50_stock[target]["stock_price"]["return_ratio"] = return_tratio

# Calculate the percentage change, and ensure columns are converted to float

In [19]:
for target in NIFTY50_stock.keys():
    function = lambda x, y: (x / y) - 1
    data = NIFTY50_stock[target]["stock_price"]

if data["OPEN "].dtype == 'object':
        data["OPEN "] = data["OPEN "].str.replace(',', '').astype(float)
if data["HIGH "].dtype == 'object':
        data["HIGH "] = data["HIGH "].str.replace(',', '').astype(float)
if data["LOW "].dtype == 'object':
        data["LOW "] = data["LOW "].str.replace(',', '').astype(float)

In [20]:
data["c_open"] = list(map(function, data["OPEN "], data["close "]))
data["c_high"] = list(map(function, data["HIGH "], data["close "]))
data["c_low"] = list(map(function, data["LOW "], data["close "]))

# Calculate 5 / 10 / 15 / 20 / 25 / 30 days moving average

In [21]:
for target in NIFTY50_stock.keys():
    data = NIFTY50_stock[target]["stock_price"]["close "]
    for i in [5, 10, 15, 20, 25, 30]:
        q = []
        for day in range(len(data)):
            if day >= i - 1:
                q.append((np.mean(data.iloc[day - i + 1:day + 1]) / data.iloc[day]) - 1)
            if day < i - 1:
                q.append(0)
        NIFTY50_stock[target]["stock_price"][f"{i}-days"] = q

# Category of sector (one hot encoding)

In [22]:
label = LabelEncoder()
label.fit(NIFTY50_name["category"].unique())

for target in NIFTY50_stock.keys():
    for lbl in NIFTY50_name["category"].unique():
        cate = NIFTY50_stock[target]['category']
        if lbl != cate:
            NIFTY50_stock[target]["stock_price"][f"label_{lbl}"] = 0
        if lbl == cate:
            NIFTY50_stock[target]["stock_price"][f"label_{lbl}"] = 1

In [16]:
NIFTY50_stock['ADANIENT']['stock_price'] = NIFTY50_stock['ADANIENT']['stock_price'].drop('nor_close', axis=1)

In [28]:
NIFTY50_stock['ADANIENT']['stock_price']

,Date,OPEN,HIGH,LOW,PREV. CLOSE,close,return_ratio
Date,,,,,,,
26-Jun-2024,26-Jun-2024,"3,169.05","3,187.70","3,163.90","3,171.10",3170.50,0.000000
25-Jun-2024,25-Jun-2024,"3,204.75","3,208.45","3,156.35","3,194.60",3171.10,0.000189
24-Jun-2024,24-Jun-2024,"3,185.75","3,214.80","3,135.00","3,189.30",3194.60,0.007411
21-Jun-2024,21-Jun-2024,"3,265.05","3,277.10","3,176.40","3,259.45",3189.30,-0.001659
20-Jun-2024,20-Jun-2024,"3,263.90","3,315.00","3,240.00","3,261.90",3259.45,0.021995
...,...,...,...,...,...,...,...
01-Jul-2022,01-Jul-2022,"2,189.90","2,242.00","2,138.25","2,190.90",2232.20,-0.013109
30-Jun-2022,30-Jun-2022,"2,220.00","2,248.50","2,180.50","2,220.10",2190.90,-0.018502
29-Jun-2022,29-Jun-2022,"2,185.00","2,247.80","2,177.20","2,204.60",2220.10,0.013328


# Total features

In [23]:
features = {}
for target in NIFTY50_stock.keys():
    features[target] = NIFTY50_stock[target]["stock_price"].iloc[30:, 7:].reset_index(drop=True)

In [30]:
features['NIFTY 50'].head()

,return_ratio
0,-0.004000
1,-0.051999
2,-0.028598
3,-0.010939
4,0.033940


In [28]:
features['ADANIENT']

,close,vwap,52W H,52W L,VOLUME,VALUE,No of trades,nor_close,return_ratio,5-days,...,20-days,25-days,30-days,label_Consumer Goods and Services,label_Healthcare and Diversified,label_Information Technology and Communication,label_Financial Services,label_ Energy and Industrials and Materials,label_Energy and Industrials and Materials,label_Financial Services
0,3037.55,"2,995.85","3,350.00","1,857.75","57,25,807","17,15,36,65,502.90","1,96,967",0.375180,-0.004000,0.004046,...,0.053555,0.056647,0.056957,0,0,0,0,0,1,0
1,2879.60,"2,839.27","3,350.00","1,857.75","37,84,373","10,74,48,38,041.40","1,15,926",0.129061,-0.051999,0.046607,...,0.105432,0.108640,0.111558,0,0,0,0,0,1,0
2,2797.25,"2,791.48","3,350.00","1,857.75","23,54,794","6,57,33,68,242.40","79,999",0.000742,-0.028598,0.058597,...,0.130427,0.134636,0.139547,0,0,0,0,0,1,0
3,2766.65,"2,814.22","3,350.00","1,857.75","14,85,510","4,18,05,48,928.95","1,16,557",-0.046939,-0.010939,0.050426,...,0.135358,0.140561,0.147059,0,0,0,0,0,1,0
4,2860.55,"2,860.65","3,350.00","1,857.75","9,51,411","2,72,16,50,623.40","57,225",0.099377,0.033940,0.002716,...,0.093635,0.098108,0.104757,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,2232.20,"2,192.41","2,420.95","1,332.75","21,41,743","4,69,55,81,540.60","47,117",-0.879724,-0.013109,0.011348,...,0.074988,0.099180,0.125285,0,0,0,0,0,1,0
463,2190.90,"2,214.04","2,420.95","1,332.75","15,68,415","3,47,25,39,125.60","46,645",-0.944078,-0.018502,0.022799,...,0.087048,0.110175,0.136248,0,0,0,0,0,1,0
464,2220.10,"2,214.49","2,420.95","1,332.75","16,92,493","3,74,80,16,616.50","52,274",-0.898578,0.013328,0.004594,...,0.065350,0.086721,0.112086,0,0,0,0,0,1,0
465,2204.60,"2,186.15","2,420.95","1,332.75","12,82,047","2,80,27,44,750.95","37,963",-0.922731,-0.006982,0.007861,...,0.064786,0.085670,0.110491,0,0,0,0,0,1,0


In [26]:
NIFTY50_stock['ADANIENT']['stock_price']['return_ratio']

,Date,series,OPEN,HIGH,LOW,PREV. CLOSE,ltp,close,vwap,52W H,...,20-days,25-days,30-days,label_Consumer Goods and Services,label_Healthcare and Diversified,label_Information Technology and Communication,label_Financial Services,label_ Energy and Industrials and Materials,label_Energy and Industrials and Materials,label_Financial Services
Date,,,,,,,,,,,,,,,,,,,,,
26-Jun-2024,26-Jun-2024,EQ,"3,169.05","3,187.70","3,163.90","3,171.10","3,169.00",3170.50,"3,174.21","3,743.90",...,0.000000,0.000000,0.000000,0,0,0,0,0,1,0
25-Jun-2024,25-Jun-2024,EQ,"3,204.75","3,208.45","3,156.35","3,194.60","3,169.05",3171.10,"3,175.93","3,743.90",...,0.000000,0.000000,0.000000,0,0,0,0,0,1,0
24-Jun-2024,24-Jun-2024,EQ,"3,185.75","3,214.80","3,135.00","3,189.30","3,193.95",3194.60,"3,191.57","3,743.90",...,0.000000,0.000000,0.000000,0,0,0,0,0,1,0
21-Jun-2024,21-Jun-2024,EQ,"3,265.05","3,277.10","3,176.40","3,259.45","3,192.80",3189.30,"3,205.27","3,743.90",...,0.000000,0.000000,0.000000,0,0,0,0,0,1,0
20-Jun-2024,20-Jun-2024,EQ,"3,263.90","3,315.00","3,240.00","3,261.90","3,261.95",3259.45,"3,279.03","3,743.90",...,0.000000,0.000000,0.000000,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
01-Jul-2022,01-Jul-2022,EQ,"2,189.90","2,242.00","2,138.25","2,190.90","2,229.25",2232.20,"2,192.41","2,420.95",...,0.074988,0.099180,0.125285,0,0,0,0,0,1,0
30-Jun-2022,30-Jun-2022,EQ,"2,220.00","2,248.50","2,180.50","2,220.10","2,190.00",2190.90,"2,214.04","2,420.95",...,0.087048,0.110175,0.136248,0,0,0,0,0,1,0
29-Jun-2022,29-Jun-2022,EQ,"2,185.00","2,247.80","2,177.20","2,204.60","2,211.00",2220.10,"2,214.49","2,420.95",...,0.065350,0.086721,0.112086,0,0,0,0,0,1,0


# Calculate movement of stock

In [31]:
Y_buy_or_not = {}
for target in NIFTY50_stock.keys():
    Y_buy_or_not[target] = (features[target]['return_ratio']>=0)*1         

# Training & testing split

In [32]:
train_size = 0.2
test_size = 0.8
days = len(features["ONGC"])

# Create data of training set and testing set

In [33]:
train_day = int(days * train_size)

train_data = {}
test_data = {}
train_Y_buy_or_not = {}
test_Y_buy_or_not = {}

In [34]:
for i in NIFTY50_stock.keys():
    train_data[i] = features[i].iloc[:train_day, :]
    train_Y_buy_or_not[i] = Y_buy_or_not[i][:train_day]
    test_data[i] = features[i].iloc[train_day:, :]
    test_Y_buy_or_not[i] = Y_buy_or_not[i][train_day:]

In [38]:
# Week represents the number of our inputs
def before_day(week):
    # Train
    train = {}
    for w in range(week):
        train_x = []
        for tr_ind in range(len(train_data["ONGC"]) - 7 - (week - 2) - 1):
            tr = []
            for target in NIFTY50_stock.keys():
                data = train_data[target]
                if tr_ind + w + 7 <= len(data):
                    tr.append(data.iloc[tr_ind + w:tr_ind + w + 7, :].values)
            if len(tr) == len(NIFTY50_stock):  # Ensure all targets are included
                train_x.append(tr)
        train[f"x{w + 1}"] = np.array(train_x, dtype=object)

    train_y1, train_y2 = [], []
    for tr_ind in range(len(train_data["ONGC"]) - 7 - (week - 2) - 1):
        tr_y1, tr_y2 = [], []
        for target in NIFTY50_stock.keys():
            data = train_data[target]
            if tr_ind + (week - 1) + 7 < len(data):
                tr_y1.append(data["return_ratio"].iloc[tr_ind + (week - 1) + 7])
                tr_y2.append(train_Y_buy_or_not[target].iloc[tr_ind + (week - 1) + 7])
        if len(tr_y1) == len(NIFTY50_stock) and len(tr_y2) == len(NIFTY50_stock):
            train_y1.append(tr_y1)
            train_y2.append(tr_y2)
    train['y_return_ratio'] = np.array(train_y1, dtype=object)
    train["y_up_or_down"] = np.array(train_y2, dtype=object)

    # Test
    test = {}
    for w in range(week):
        test_x = []
        for te_ind in range(len(test_data["ONGC"]) - 7 - (week - 2) - 1):
            te = []
            for target in NIFTY50_stock.keys():
                data = test_data[target]
                if te_ind + w + 7 <= len(data):
                    te.append(data.iloc[te_ind + w:te_ind + w + 7, :].values)
            if len(te) == len(NIFTY50_stock):  # Ensure all targets are included
                test_x.append(te)
        test[f'x{w + 1}'] = np.array(test_x, dtype=object)

    test_y1, test_y2 = [], []
    for te_ind in range(len(test_data["ONGC"]) - 7 - (week - 2) - 1):
        te_y1, te_y2 = [], []
        for target in NIFTY50_stock.keys():
            data = test_data[target]
            if te_ind + (week - 1) + 7 < len(data):
                te_y1.append(data["return_ratio"].iloc[te_ind + (week - 1) + 7])
                te_y2.append(test_Y_buy_or_not[target].iloc[te_ind + (week - 1) + 7])
        if len(te_y1) == len(NIFTY50_stock) and len(te_y2) == len(NIFTY50_stock):
            test_y1.append(te_y1)
            test_y2.append(te_y2)
    test['y_return_ratio'] = np.array(test_y1, dtype=object)
    test["y_up_or_down"] = np.array(test_y2, dtype=object)

    data = {"train": train, "test": test}

    return data

# Convert strings to float

In [40]:
import pickle
import numpy as np
import pandas as pd

def str_to_float(value):
    if isinstance(value, str):
        try:
            return float(value)
        except ValueError:
            return value
    return value

In [66]:
# Load the pickle file
with open('D:\\capstone_project\\capstone_project\\NIFTY50_dataaa_final_pls.pkl', 'rb') as f:
    dt = pickle.load(f)

In [67]:
dt['train'].keys()

dict_keys(['x1', 'x2', 'x3', 'x4', 'y_return_ratio', 'y_up_or_down'])

In [68]:
def convert_to_int(s):
    # Remove comma
    s = s.replace(',', '')
    
    # Round to nearest integer and convert to int
    return np.float64(s)

In [228]:
# print(converted_data['train'].keys())

dict_keys(['x1', 'x2', 'x3', 'x4', 'y_return_ratio', 'y_up_or_down'])


In [69]:
dt['train']['x1']

array([[[[2362.55, '2,352.04', '2,769.65', ..., 0, 0, 0],
         [2325.65, '2,332.54', '2,769.65', ..., 0, 0, 0],
         [2341.05, '2,340.63', '2,769.65', ..., 0, 0, 0],
         ...,
         [2256.2, '2,239.72', '2,769.65', ..., 0, 0, 0],
         [2217.05, '2,214.90', '2,769.65', ..., 0, 0, 0],
         [2226.3, '2,232.05', '2,769.65', ..., 0, 0, 0]],

        [[2533.2, '2,534.73', '27,693.00', ..., 0, 0, 0],
         [2513.45, '2,514.86', '27,693.00', ..., 0, 0, 0],
         [2539.0, '2,522.54', '27,693.00', ..., 0, 0, 0],
         ...,
         [2457.4, '2,459.83', '27,693.00', ..., 0, 0, 0],
         [2456.1, '2,466.45', '27,693.00', ..., 0, 0, 0],
         [2512.3, '2,522.60', '27,693.00', ..., 0, 0, 0]],

        [[4877.4, '4,893.64', '4,954.35', ..., 0, 0, 0],
         [4764.9, '4,815.58', '4,949.05', ..., 0, 0, 0],
         [4613.9, '4,560.11', '4,949.05', ..., 0, 0, 0],
         ...,
         [4509.75, '4,508.61', '4,949.05', ..., 0, 0, 0],
         [4552.75, '4,549.85',

In [71]:
A = ['train', 'test']
B = ['x1', 'x2', 'x3', 'x4']

for a in A:
    for b in B:
        x,y,z, w = dt[a][b].shape
        for i in range(x):
            for j in range(y):
                for k in range(z):
                    for l in range(w):
                        if(type(dt[a][b][i][j][k][l])==str):
                            dt[a][b][i][j][k][l] = convert_to_int(dt[a][b][i][j][k][l])
                        else:
                            dt[a][b][i][j][k][l] = np.float64(dt[a][b][i][j][k][l])

In [72]:
A = ['train', 'test']
B = ['y_return_ratio', 'y_up_or_down']


for a in A:
    for b in B:
        x,y = dt[a][b].shape
        for i in range(x):
            for j in range(y):
                        if(type(dt[a][b][i][j])!=np.float64):
                              dt[a][b][i][j] = np.float64(dt[a][b][i][j])

In [74]:
type(dt['test']['y_up_or_down'][0][0])

numpy.float64

In [75]:
lis = []

In [76]:
a = 'test'
b = 'y_return_ratio'
x,y = dt[a][b].shape
for i in range(x):
        for j in range(y):
            if(type(dt[a][b][i][j])!=np.float64):
                tem = []
                tem.append(i)
                tem.append(j)
                lis.append(tem)

In [217]:
# x,y = len(lis), len(lis[0])

In [224]:
# a = 'test'
# b = 'y_up_or_down'

# for q in lis:
#     df[a][b][q[0]][q[1]] = np.float64(df[a][b][q[0]][q[1]])


In [214]:
# len(lis[0])

2

In [95]:
# for a in list(df.keys()):
#     for b in list(df[a].keys()):
#         x,y,z,w = df[a][b].shape
#         for i in range(x):
#             for j in range(y):
#                 for k in range(z):
#                     for l in range(w):
#                         if(type(df[a][b][i][j][k][l])==str):
#                             df[a][b][i][j][k][l] = convert_to_int(df[a][b][i][j][k][l])
#                         elif(type(df[a][b][i][j][k][l])==int):
#                             df[a][b][i][j][k][l] = float(df[a][b][i][j][k][l])
#         print(a, b)

train x1
train x2
train x3
train x4


ValueError: not enough values to unpack (expected 4, got 2)

In [56]:
# for i in range(df.shape[0]):
#     for j in range(df.shape[1]):
#         for k in range(df.shape[2]):
#             for l in range(df.shape[3]):
#                 if(type(df[i][j][k][l])==str):
#                     df[i][j][k][l] = convert_to_int(df[i][j][k][l])
#                 elif(type(df[i][j][k][l])==int):
#                     df[i][j][k][l] = float(df[i][j][k][l])

In [77]:
with open('converted_pickle_file2_final.pkl', 'wb') as f:
    pickle.dump(dt, f)

print("Conversion complete. New file saved as 'converted_pickle_file2puka.pkl'")

Conversion complete. New file saved as 'converted_pickle_file2puka.pkl'


In [ ]:
# # Save the converted data back to a pickle file
# with open('converted_pickle_file.pkl', 'wb') as f:
#     pickle.dump(converted_data, f)

# print("Conversion complete. New file saved as 'converted_pickle_file.pkl'")

In [25]:
# Example usage
data1 = data

In [235]:
data1 = before_day(4)

ValueError: could not broadcast input array from shape (7,1) into shape (7,)

In [27]:
data1.shape

(497,)

In [3]:
# Example usage
data = before_day(4)

# Save the processed data to a pickle file
with open('NIFTY50_data.pkl', 'wb') as f:
    pickle.dump(data, f)

print("code is succesfully collected..")

code is succesfully collected..
